In [ ]:
# Cell 1: Import necessary libraries
from nilearn.maskers import NiftiLabelsMasker
from scipy.ndimage import center_of_mass
import nibabel as nib
import cupy as cp  # GPU-accelerated NumPy
import pandas as pd  # Replace with cudf or modin if needed
from joblib import Parallel, delayed  # For parallel processing
import matplotlib.pyplot as plt
import os

In [ ]:
# Cell 2: Define constants
data_dir = '/home/zaz22/research-data-store/fmri/fmri_beijing'
mapping_file = '/home/zaz22/research-data-store/rois/rois_3000_beijing/rois/brain_atoms.mnc.gz'  
repetition_time = 2.0

# List of subject IDs
subject_ids = [
    '9640133', '9783279', '9887336', '9890726', '4095748', '4136226', 
    '4221029', '4225073', '4241194', '4256491', '4265987', '4334113',
    '4383707', '4475709', '4921428', '5150328', '5193577', '5575344', 
    '5600820', '5669389', '5993008', '6187322', '6383713', '6477085',
    '6500128', '7011503', '7093319', '7135128', '7253183', '7390867', 
    '7407032', '7689953', '7994085', '8191384', '8278680',
]

In [ ]:
# Cell 3: Load ROI mapping file
mapping_img = nib.load(mapping_file)
mapping_data = cp.array(mapping_img.get_fdata())  # Use CuPy for GPU acceleration
affine = mapping_img.affine

# Extract unique ROI labels, excluding 0
roi_labels = cp.unique(mapping_data)
roi_labels = roi_labels[roi_labels != 0]
print(f'Number of ROIs: {roi_labels.size}')

In [ ]:
# Cell 4: Define a function to process a single subject
def process_subject(subject_id):
    """
    Load and process fMRI data for a single subject.
    """
    # Load subject-specific fMRI data
    subject_file = os.path.join(data_dir, f"sub-{subject_id}_bold.nii.gz")
    if not os.path.exists(subject_file):
        print(f"File {subject_file} not found. Skipping...")
        return None
    
    # Load fMRI data using nibabel
    fmri_img = nib.load(subject_file)
    fmri_data = cp.array(fmri_img.get_fdata())  # Use CuPy for GPU acceleration

    # Perform subject-specific processing (e.g., extract timeseries)
    timeseries = cp.mean(fmri_data, axis=(0, 1, 2))  # Example operation
    
    return timeseries


In [ ]:
# Cell 5: Process all subjects in parallel
results = Parallel(n_jobs=-1, backend="multiprocessing")(
    delayed(process_subject)(subject_id) for subject_id in subject_ids
)

# Convert results into a DataFrame for analysis
df = pd.DataFrame(results, index=subject_ids)
df.columns = [f"Timepoint_{i}" for i in range(df.shape[1])]
print(df.head())


In [ ]:
# Cell 6: Analyze and visualize results
def plot_average_timeseries(dataframe):
    """
    Plot the average timeseries across all subjects.
    """
    mean_timeseries = dataframe.mean(axis=0)
    
    # Plotting
    plt.figure(figsize=(12, 6))
    plt.plot(mean_timeseries, marker='o', linestyle='-', label='Average Timeseries')
    plt.title('Average fMRI Timeseries Across Subjects', fontsize=16)
    plt.xlabel('Timepoints', fontsize=14)
    plt.ylabel('Signal Intensity', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend(fontsize=12)
    plt.show()

# Call the visualization function
plot_average_timeseries(df)